# Fine-tuned Phi-4 Model with Unsloth

This model was fine-tuned using the Phi-4 architecture and the Unsloth library.  
The dataset was converted into the appropriate format for instruction tuning.  
The final model is saved and stored in Google Drive for future use.


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [2]:
!pip install -U transformers --upgrade

### Unsloth

In [3]:
from unsloth import FastLanguageModel  # FastVisionModel for LLMs
import torch
max_seq_length = 2048  # Choose any! We auto support RoPE Scaling internally!
load_in_4bit = True  # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Phi-4",  # Phi-4 2x faster!
    "unsloth/Phi-4-unsloth-bnb-4bit",  # Phi-4 Unsloth Dynamic 4-bit Quant
]  # More models at https://docs.unsloth.ai/get-started/all-our-models

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-4",
    max_seq_length = max_seq_length,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

We now add LoRA adapters for parameter efficient finetuning - this allows us to only efficiently train 1% of all parameters.

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.5.1 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the `Phi-4` format for conversation style finetunes. But we convert it to HuggingFace's normal multiturn format `("role", "content")` instead of `("from", "value")`/ Phi-4 renders multi turn conversations like below:

```
<|im_start|>user<|im_sep|>Hello!<|im_end|>
<|im_start|>assistant<|im_sep|>Hi! How can I help?<|im_end|>
<|im_start|>user<|im_sep|>What is 2+2?<|im_end|>
```


In [5]:
import pandas as pd

df = pd.read_excel("/content/dataset.xlsx")


In [6]:
import pandas as pd
import json


# Her soru-cevap çiftini formatla ve JSONL satırı olarak hazırla
lines = []
for _, row in df.iterrows():
    question = str(row['Question']).strip()
    answer = str(row['Answer']).strip()
    chat_format = f"<|im_start|>user<|im_sep|>{question}<|im_end|><|im_start|>assistant<|im_sep|>{answer}<|im_end|>"
    lines.append({"text": chat_format})

# JSONL formatında kaydetmek istersen:
with open("phi4_dataset.jsonl", "w", encoding="utf-8") as f:
    for line in lines:
        f.write(json.dumps(line, ensure_ascii=False) + "\n")

# Değişken olarak da kullanabilirsin
dataset = lines


In [7]:
dataset

[{'text': '<|im_start|>user<|im_sep|>What is the main purpose of Power Apps within the Power Platform?<|im_end|><|im_start|>assistant<|im_sep|>To allow users to build low-code/no-code custom business applications.<|im_end|>'},
 {'text': '<|im_start|>user<|im_sep|>How do Power Apps and Power Automate complement each other in the Power Platform?<|im_end|><|im_start|>assistant<|im_sep|>Power Apps is used to build apps, while Power Automate enables automation of workflows within those apps.<|im_end|>'},
 {'text': "<|im_start|>user<|im_sep|>What does 'within app context' mean in Power Automate licensing?<|im_end|><|im_start|>assistant<|im_sep|>It means flows can only access data and services directly related to the app that triggered them.<|im_end|>"},
 {'text': '<|im_start|>user<|im_sep|>What is the difference between attended and unattended RPA in Power Automate?<|im_end|><|im_start|>assistant<|im_sep|>Attended RPA runs with user interaction, while unattended RPA runs autonomously.<|im_en

In [8]:
dataset[0]["text"]

'<|im_start|>user<|im_sep|>What is the main purpose of Power Apps within the Power Platform?<|im_end|><|im_start|>assistant<|im_sep|>To allow users to build low-code/no-code custom business applications.<|im_end|>'

In [9]:
from datasets import Dataset

# JSONL olarak oluşturduğun listeyi HuggingFace Dataset'e dönüştür
hf_dataset = Dataset.from_list(dataset)


<a name="Train"></a>
### Train the model


In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = hf_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/98 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [11]:
from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part="<|im_start|>user<|im_sep|>",
    response_part="<|im_start|>assistant<|im_sep|>",
)

Map (num_proc=2):   0%|          | 0/98 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by promote_options='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by promote_options='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


We verify masking is actually done:

In [12]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|im_start|>user<|im_sep|>Why would an organization use Power Apps per app license instead of Premium?<|im_end|><|im_start|>assistant<|im_sep|>For limited use cases where only one or two apps need to be run per user.<|im_end|>'

In [13]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                     For limited use cases where only one or two apps need to be run per user.<|im_end|>'

We can see the System and Instruction prompts are successfully masked!

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [14]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 98 | Num Epochs = 3 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 65,536,000/4,000,000,000 (1.64% trained)


Step,Training Loss
1,5.832700
2,5.844000
3,5.061700
4,4.808300
5,4.602300
6,3.975900
7,3.100900
8,2.570400
9,2.808200
10,2.864000


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Unsloth_Studio.ipynb)**

We use `min_p = 0.1` and `temperature = 1.5`.

In [15]:
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel
import torch
import re

tokenizer = get_chat_template(tokenizer, chat_template = "phi-4")
FastLanguageModel.for_inference(model)

def parse_phi_chat(text):
    pattern = r"<\|im_start\|>(user|assistant)<\|im_sep\|>(.*?)<\|im_end\|>"
    matches = re.findall(pattern, text, re.DOTALL)
    return [{"role": role.strip(), "content": content.strip()} for role, content in matches]

text = dataset[0]["text"]
messages = parse_phi_chat(text)

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda" if torch.cuda.is_available() else "cpu")

outputs = model.generate(
    input_ids = inputs,
    max_new_tokens = 100,
    use_cache = True,
    temperature = 1.0,
    top_p = 0.95,
)

#Sadece üretilen kısmı decode et
generated = outputs[0][inputs.shape[-1]:]
print(tokenizer.decode(generated, skip_special_tokens=True))


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Power Automate is used for automating workflows and business processes.


 You can also use a `TextStreamer` for continuous inference - so you can see The generation token by token, instead of waiting the whole time!

## 🧪 Inference on Custom Dataset and Exporting Model Outputs to CSV

This section performs **inference using a fine-tuned language model** (e.g., Phi-4 with LoRA adapters) on a custom dataset containing chat-style prompts and expected responses. The model's outputs are collected and saved into a CSV file for further analysis.

### 📝 Dataset Format
Each entry in the dataset is a single string with special formatting using tokens:

<|im_start|>user<|im_sep|>QUESTION<|im_end|>

<|im_start|>assistant<|im_sep|>EXPECTED_ANSWER<|im_end|>



### 🎯 Goal
- Extract the `QUESTION` part from each dataset item.
- Generate a response using the model.
- Compare it with the `EXPECTED_ANSWER` (optional).
- Save all results into a `model_outputs.csv` file.

### 📤 Output CSV Columns
- **question**: User input extracted from the dataset.
- **expected_answer**: The reference assistant answer.
- **generated_answer**: The model’s generated response.

This step is essential for validating how well the model performs on custom examples and helps in error analysis or evaluation.


In [16]:
import torch
import openpyxl
from transformers import TextStreamer
from openpyxl import Workbook

results = []

for i, row in enumerate(dataset):
    text = row["text"]

    # Soruyu çıkar
    try:
        question = text.split("<|im_sep|>")[1].split("<|im_end|>")[0].strip()
    except:
        print(f"Parse hatası satır {i}: {text}")
        continue

    messages = [{"role": "user", "content": question}]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")

    # Modelden yanıt al
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs,
            max_new_tokens=128,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            use_cache=True,
        )

    # Tokenleri çöz
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=False)

    # Sadece assistant cevabını al
    if "<|im_start|>assistant<|im_sep|>" in decoded_output:
        generated_answer = decoded_output.split("<|im_start|>assistant<|im_sep|>")[-1].split("<|im_end|>")[0].strip()
    else:
        generated_answer = decoded_output.strip()

    # Referans cevabı al
    try:
        expected_answer = text.split("<|im_start|>assistant<|im_sep|>")[1].split("<|im_end|>")[0].strip()
    except:
        expected_answer = ""

    results.append((question, expected_answer, generated_answer))


wb = Workbook()
ws = wb.active
ws.title = "Model Cevapları"

ws.append(["question", "expected_answer", "generated_answer"])

# Datas
for row in results:
    ws.append(row)

wb.save("model_outputs.xlsx")
print("✅ Excel dosyası 'model_outputs.xlsx' olarak kaydedildi.")


✅ Excel dosyası 'model_outputs.xlsx' olarak kaydedildi.


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [17]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/vocab.json',
 'lora_model/merges.txt',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [18]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "How can users customize Copilot behavior?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(
    input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
    use_cache = True, temperature = 1.5, min_p = 0.1
)

By adjusting settings and preferences.<|im_end|>


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [19]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer

    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model",  # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit=load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

#### Google Drive Connect

In [20]:
from google.colab import drive
import shutil
import os

# 1. Google Drive'ı mount et
drive.mount('/content/drive')

# 2. Kaynak klasör (modelin bulunduğu yer)
source_folder = "/content/lora_model"  # lora_model klasörün buradaysa
drive_folder = "/content/drive/MyDrive/fine_tuned_phi4_2"
os.makedirs(drive_folder, exist_ok=True)

# 3. Kopyalanacak dosyaların listesi
files_to_copy = [
    "adapter_model.safetensors",
    "adapter_config.json",
    "tokenizer.json",
    "tokenizer_config.json",
    "vocab.json",
    "merges.txt",
    "special_tokens_map.json"
]

# 4. Dosyaları kopyala
for file_name in files_to_copy:
    source_path = os.path.join(source_folder, file_name)
    if os.path.exists(source_path):
        shutil.copy(source_path, drive_folder)
        print(f"✅ {file_name} Drive'a kopyalandı.")
    else:
        print(f"⚠️ {file_name} bulunamadı, atlandı.")

print("📁 Tüm işlemler tamamlandı.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ adapter_model.safetensors Drive'a kopyalandı.
✅ adapter_config.json Drive'a kopyalandı.
✅ tokenizer.json Drive'a kopyalandı.
✅ tokenizer_config.json Drive'a kopyalandı.
✅ vocab.json Drive'a kopyalandı.
✅ merges.txt Drive'a kopyalandı.
✅ special_tokens_map.json Drive'a kopyalandı.
📁 Tüm işlemler tamamlandı.
